In [ ]:
# 安裝必要套件
!pip install langgraph openai


In [ ]:
# 導入模組
from langgraph.graph import StateGraph, END
from dataclasses import dataclass
from typing import Dict
from openai import OpenAI
from google.colab import userdata

# 定義狀態結構
@dataclass
class ReflectionState:
    input: str
    eng: str
    jpn: str
    kor: str
    feedback: str = ""
    has_issue: bool = False

# 初始化OpenAI客戶端
client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))  # 替換為您的API金鑰

# 語言模型調用函數
def llm_call(prompt: str, max_tokens: int = 200) -> str:
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=max_tokens
    )
    return response.choices[0].message.content.strip()

# 反思節點：檢查語義一致性
def reflect_node(state: ReflectionState) -> Dict:
    prompt = f"檢查以下翻譯是否語義一致：\n中文：{state.input}\n英文：{state.eng}\n日語：{state.jpn}\n韓語：{state.kor}\n若有問題，回覆具體建議；若無，回覆'無問題'"
    state.feedback = llm_call(prompt)
    state.has_issue = "無問題" not in state.feedback
    return {"feedback": state.feedback, "has_issue": state.has_issue}

# 更新節點：修正翻譯偏差
def update_node(state: ReflectionState) -> Dict:
    prompt = f"根據建議修正翻譯：\n建議：{state.feedback}\n原文：{state.input}\n當前：英文-{state.eng}，日語-{state.jpn}，韓語-{state.kor}\n回覆格式：英文;日語;韓語"
    response = llm_call(prompt)
    try:
        eng, jpn, kor = response.split(";")
        return {"eng": eng.strip(), "jpn": jpn.strip(), "kor": kor.strip()}
    except ValueError:
        return {"eng": state.eng, "jpn": state.jpn, "kor": state.kor}  # 防止格式錯誤

# 構建工作流程
workflow = StateGraph(ReflectionState)
workflow.add_node("reflect", reflect_node)
workflow.add_node("update", update_node)
workflow.set_entry_point("reflect")
workflow.add_conditional_edges(
    "reflect",
    lambda state: "update" if state.has_issue else "end",
    {"update": "update", "end": END}
)
workflow.add_edge("update", "reflect")
graph = workflow.compile()

# 執行範例
result = graph.invoke(ReflectionState(
    input="人工智慧改變未來",
    eng="AI changes the future",  # 故意設為不準確
    jpn="人工知能が未来を変える",
    kor="인공지능이 미래를 바꾼다"
))
print(f"最終翻譯：\n英文：{result['eng']}\n日語：{result['jpn']}\n韓語：{result['kor']}")
print(f"反饋：{result['feedback']}")